## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
# gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,66.04,71,0,12.26,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,16.11,broken clouds
2,2,Saskylakh,RU,71.9167,114.0833,-17.90,89,46,3.00,scattered clouds
3,3,Georgetown,MY,5.4112,100.3354,79.90,90,40,3.44,scattered clouds
4,4,Manaure,CO,11.7751,-72.4445,89.71,59,10,22.79,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Georgetown,MY,5.4112,100.3354,79.90,90,40,3.44,scattered clouds
4,4,Manaure,CO,11.7751,-72.4445,89.71,59,10,22.79,clear sky
7,7,Kapaa,US,22.0752,-159.3190,82.38,60,40,16.11,scattered clouds
9,9,Carnarvon,AU,-24.8667,113.6333,77.07,88,3,0.00,clear sky
10,10,Pedro Juan Caballero,PY,-22.5472,-55.7333,80.85,37,96,3.94,overcast clouds
12,12,Takoradi,GH,4.8845,-1.7554,78.21,87,68,6.04,light rain
13,13,Dingle,PH,10.9995,122.6711,75.51,91,97,6.53,overcast clouds
18,18,Kihei,US,20.7850,-156.4656,80.62,66,20,17.27,light rain
20,20,Avarua,CK,-21.2078,-159.7750,82.45,74,20,4.61,light rain
24,24,Butaritari,KI,3.0707,172.7902,80.80,79,68,12.53,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

182


Unnamed: 0             182
City                   182
Country                182
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

182


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Georgetown,MY,5.4112,100.3354,79.90,90,40,3.44,scattered clouds
4,4,Manaure,CO,11.7751,-72.4445,89.71,59,10,22.79,clear sky
7,7,Kapaa,US,22.0752,-159.3190,82.38,60,40,16.11,scattered clouds
9,9,Carnarvon,AU,-24.8667,113.6333,77.07,88,3,0.00,clear sky
10,10,Pedro Juan Caballero,PY,-22.5472,-55.7333,80.85,37,96,3.94,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,79.90,scattered clouds,5.4112,100.3354,
4,Manaure,CO,89.71,clear sky,11.7751,-72.4445,
7,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
9,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
10,Pedro Juan Caballero,PY,80.85,overcast clouds,-22.5472,-55.7333,
12,Takoradi,GH,78.21,light rain,4.8845,-1.7554,
13,Dingle,PH,75.51,overcast clouds,10.9995,122.6711,
18,Kihei,US,80.62,light rain,20.7850,-156.4656,
20,Avarua,CK,82.45,light rain,-21.2078,-159.7750,
24,Butaritari,KI,80.80,moderate rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,79.90,scattered clouds,5.4112,100.3354,Cititel Penang
4,Manaure,CO,89.71,clear sky,11.7751,-72.4445,Hotel Palaaima
7,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,Hospitality Carnarvon
10,Pedro Juan Caballero,PY,80.85,overcast clouds,-22.5472,-55.7333,Hotel Internacional
12,Takoradi,GH,78.21,light rain,4.8845,-1.7554,Raybow International Hotel
13,Dingle,PH,75.51,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
18,Kihei,US,80.62,light rain,20.7850,-156.4656,Maui Sunset
20,Avarua,CK,82.45,light rain,-21.2078,-159.7750,Paradise Inn
24,Butaritari,KI,80.80,moderate rain,3.0707,172.7902,Isles Sunset Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))